# MLP-12 Numpy Model Extraction

**Goals:**

- Load Dataset and Model, then verify.
- Extract the weights.
- Describe the model using tensor operations, then validate.
- Describe the model using numpy matrix operations, then validate.
- Export numpy model as sqlite3 database for implementation in C.
- Export test dataset as sqlite3 database for implementation in C.


**NOTE:** The dataset exported by the training notebook may have incorrect predicted index due to several iterations of model training and not updating the dataset. We'll re-run the predictions here and update the predicted index in the dataset.

# Environment Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import IPython.display as ipd
from tqdm.auto import tqdm


# We don't need GPU for this, not training
#if torch.cuda.is_available():
#    device = torch.device('cuda')
#else:
#    device = torch.device('cpu')

device = 'cpu'
print('Using PyTorch version:', torch.__version__, ' Device:', device)

# Load and Validate torch.nn.Module Implementation

## Define Model

**NOTE:** Always copy the following cell from the training notebook.

In [ ]:
# Define an MLP with single hiddend layer with 12 units and ReLU activation.
class MLP12(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP12, self).__init__()
        # Save parameters
        self.input_size = input_size
        self.num_classes = num_classes
        self.debug = False    # can be used to activate debugging features
        # Define layers
        self.fc1 = nn.Linear(input_size, 12)   # 12 hidden units
        self.fc1_drop = nn.Dropout(0.2)        # drop-out for faster training, has no effect on inference
        self.fc2 = nn.Linear(12, num_classes)  # output layer

    # Expects a batch of 1-D tensor
    # Dimension of x: (batch-size, input_size)
    def forward(self, x):
        x = F.relu(self.fc1(x))   # pass through the hidden layer
        x = self.fc1_drop(x)      
        x = self.fc2(x)           # pass through the output layer
        return x

## Load Saved Model

In [ ]:
!ls -ltr ./saved/
print('')

# Loaded saved model dictionary
model_path = './saved/trained_mlp12-94.0p.pt'
model_dict = torch.load(model_path)
print(model_dict.keys())
for k,v in model_dict.items():
    if k!='state_dict': print(k,':',v)
        
        
# Parse the values for easier use
Accuracy = model_dict['accuracy']
Correct_count = model_dict['correct_count']
Hparam = model_dict['Hparam']
Model_state_dict = model_dict['state_dict']
Model_perf = f'Model Performance:   accuracy: {Accuracy:.2f}%   correct_count: {Correct_count}'  # to be used later
print('Hparam:', Hparam)
print('Model_perf:', Model_perf)


# move all weights to cpu
for key in Model_state_dict: 
    Model_state_dict[key] = Model_state_dict[key].to('cpu')
    

# Instantiate the model
model_pt = MLP12(Hparam['input_size'], Hparam['num_classes'])
model_pt.load_state_dict(Model_state_dict)
model_pt.to('cpu')
model_pt.eval()     # we are always evaluating here
print(model_pt)

In [ ]:
del model_path, model_dict

## Load Saved Dataset

In [ ]:
# Prints a dataset item
def print_dataitem(item):
    mstr = f"label: {item[0]}, label_index: {item[1]}, predicted_index: {item[2]}, feature_length: {item[3]},"
    mstr2 = f"feature_vector size: {len(item[4])}"
    print(mstr, mstr2)

    
# Load the test dataset
ds_path = './saved/test_dataset.pt'
DS_loaded = torch.load(ds_path)
for key in DS_loaded:
    if key != 'dataset':
        print(f'{key}:', DS_loaded[key])

        
# Show an item summary
item = DS_loaded['dataset'][0]
print('item-> ', end='')
print_dataitem(item)

In [ ]:
del ds_path, item, key

## Validate The Loaded Model

In [ ]:
# find most likely label index for each element
def get_likely_index(tensor):
    # convert to tensor from numpy if needed
    if not torch.is_tensor(tensor):
        tensor = torch.from_numpy(tensor)
    return tensor.argmax(dim=-1)


# Given an item form the test_dataset, returns an example for predict() function
# numpytype: set it to True to return numpy nd-array
def make_example(data_item, numpytype=False):
    feature = torch.tensor(data_item[4])
    if numpytype: feature = feature.detach().numpy()
    return feature


# test prediction from dataset item.
# ptmodel: set it to True for the PyTorch model
def predict(example, model=None, ptmodel=False):  # example: feature_vector
    if ptmodel: model.eval()    # set the pytorch model to evaluation mode
    # Use the model to predict the label of the image
    feature = example
    if ptmodel: feature = feature.unsqueeze(0)    # add the batch dimension for the pytorch model
    output = model(feature)
    pred = get_likely_index(output)
    if ptmodel: pred = pred[0]    # removing batch index
    return pred.item()


# Test predict()
item = DS_loaded['dataset'][0]
example = make_example(item)
pred = predict(example, model=model_pt, ptmodel=True)
print('pred:',pred)
print_dataitem(item)

# Delete names
del item, example, pred

In [ ]:
# Validate the Given model on the whole dataset
# ptmodel: set it to True for the PyTorch model
def validateModel(model=None, ptmodel=False, numpytype=False):
    dataset = DS_loaded['dataset']
    expect_miss = 0      # keeps track of no. of mismatche between prediction in dataset vs model prediction
    total_count = 0
    correct_count = 0
    for item in tqdm(dataset):
        lbl, lbl_index, pred_index, *_ = item
        example = make_example(item, numpytype=numpytype)
        pred = predict(example, model=model, ptmodel=ptmodel)
        if pred != pred_index: expect_miss += 1    # prediction does not match prediction in dataset
        if pred == lbl_index: correct_count += 1   # prediction matched the actual label-index
        total_count += 1
    # Compute and print statistics
    accuracy = (100.0 * correct_count) / total_count
    print(f'Validation accuracy: {accuracy:.2f}%   correct_count: {correct_count}   expected-miss: {expect_miss}   total_count: {total_count}')
    return accuracy, correct_count, expect_miss, total_count

            
# Validate the loaded model
validateModel(model_pt, ptmodel=True)
print('Expected', Model_perf)

# Implementation Using torch.tensor Operations

In [ ]:
# Extract the weights as torch.tensors
for key in Model_state_dict:
    print(f'{key:10}:', Model_state_dict[key].size())

fc1_weight_pt = Model_state_dict['fc1.weight']
fc1_bias_pt = Model_state_dict['fc1.bias']
fc2_weight_pt = Model_state_dict['fc2.weight']
fc2_bias_pt = Model_state_dict['fc2.bias']

In [ ]:
# Define the model using pytorch tensor operations.
# Input interface is the same as the 
def tensorModel(features):
    x1 = fc1_weight_pt @ features + fc1_bias_pt
    fc1_out = F.relu(x1)
    fc2_out = fc2_weight_pt @ fc1_out + fc2_bias_pt
    return fc2_out


# Test this model
item = DS_loaded['dataset'][0]
example = make_example(item)
pred = predict(example, model=tensorModel, ptmodel=False)
print('pred:',pred)
print_dataitem(item)

# Delete names
del item, example, pred

In [ ]:
# Validate the tensor operation based model
validateModel(tensorModel, ptmodel=False)
print('Expected', Model_perf)

# Implement Using Numpy Matrix Operations

In [ ]:
# Copy weights as numpy ndarray
fc1_weight_np = fc1_weight_pt.detach().numpy()
fc1_bias_np   = fc1_bias_pt.detach().numpy()
fc2_weight_np = fc2_weight_pt.detach().numpy()
fc2_bias_np   = fc2_bias_pt.detach().numpy()

print('fc1_weight_np:', fc1_weight_np.shape)

In [ ]:
# Relu on numpy array
def npReLU(np_arr):
    return np.maximum(0, np_arr)


# Define the model using numpy matrix operations
# Input interface is the same as the 
def numpyModel(features):
    x1 = fc1_weight_np @ features + fc1_bias_np
    fc1_out = npReLU(x1)
    fc2_out = fc2_weight_np @ fc1_out + fc2_bias_np
    return fc2_out


# Test this model
item = DS_loaded['dataset'][0]
example = make_example(item, numpytype=True)
pred = predict(example, model=numpyModel, ptmodel=False)
print('pred:',pred)
print_dataitem(item)

# Delete names
del item, example, pred

In [ ]:
# Validate the tensor operation based model
validateModel(numpyModel, ptmodel=False, numpytype=True)
print('Expected', Model_perf)

# Update the dataset with the Numpy Model Predicted index

In [ ]:
enum_iter = tqdm( enumerate(DS_loaded['dataset']), total=len(DS_loaded['dataset']) )
fix_count = 0
for index, item in enum_iter:
    # Make prediction using Numpy model
    example = make_example(item, numpytype=True)
    pred = predict(example, model=numpyModel, ptmodel=False)
    # Check and fix the predicted_index in the dataset
    if pred!=item[2]:
        DS_loaded['dataset'][index][2] = pred
        fix_count += 1

print(f'INFO: Fixed {fix_count} predicted_index in the dataset')

# Delete names
del enum_iter, fix_count, index, item, example, pred

# Export Numpy Model as sqlite3 DB

In [ ]:
# Delete the cache and import sqlite3 utilities
!rm -rf __pycache__/
from utilsqlite3 import *

In [ ]:
# Create the database file
DB_path = './saved/trained-mlp12.s3db'
createDB(DB_path, overwrite=True)
!ls -ltrh ./saved

## Write the header table

In [ ]:
# Creates the header table
def createHeaderTable(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Create the table
    query_str = '''CREATE TABLE IF NOT EXISTS Header (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        key TEXT,
                        value NUMERIC,
                        description TEXT
                    )'''
    cursor.execute(query_str)
    # Commit the changes and close the connection
    conn.commit()
    conn.close()


# Create the table and check 
createHeaderTable(DB_path)
table_names = getTableNames(DB_path)
print(table_names)
del table_names

In [ ]:
# Inserts a record into the Header table (will be called by insertRecordList() utility function)
def insertHeaderRecord(cursor, record):
    key, value, description = record  # this serves as a soft check for the record
    # Insert the record into the table
    cursor.execute('''INSERT INTO Header (key, value, description)
                      VALUES (?, ?, ?)''', (key, value, description))


# Call the function to insert a record
insertRecordList(DB_path, insertHeaderRecord, [('example_key', 'example_value', 'example_description')])
getRecords(DB_path, 'Header')

In [ ]:
# Define the records as a list
Fc1w_table = 'FC1_Weight_T'
Fc1b_table = 'FC1_Bias_T'
Fc2w_table = 'FC2_Weight_T'
Fc2b_table = 'FC2_Bias_T'
Hparam_table = 'Hparam_T'

header_records = [
    ('name', 'MLP-12', ''),
    ('architecture', '784-FC:12-10', 'It is an MLP with 1 hidden layer with 12 units with ReLU activation. Trained on MNIST dataset (output layer with 10 units).'),
    ('accuracy', Accuracy, 'Accuracy% of the trained model on the test dataset.'),
    ('correct_count', Correct_count, 'Number of correct predictions by the trained model on the test dataset.'),
    
    ('Hparam.table',   Hparam_table, 'This is the name of the table that contains different parameters of the model.'),
    ('fc1.weight.table', Fc1w_table, 'Name of the table containing the fc1.weight matrix'),
    ('fc1.bias.table',   Fc1b_table, 'Name of the table containing the fc1.bias vector'),
    ('fc2.weight.table', Fc2w_table, 'Name of the table containing the fc2.weight matrix'),
    ('fc2.bias.table',   Fc2b_table, 'Name of the table containing the fc2.bias vector'),
]

# Insert the header records
deleteRows(DB_path, 'Header')  # delete previous records
insertRecordList(DB_path, insertHeaderRecord, header_records)
records = getRecords(DB_path, 'Header')
for r in records: print(r[:-1])   # print all but description field

# Export the Dataset as sqlite3 DB

# Concluding Remarks